In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [38]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [39]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [40]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [41]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [47]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [48]:
course_data_df.rename(columns={
    'ID_Course':'ID_Course__c',
    'CourseCode':'CourseCode__c',
    'CourseName':'CourseName__c',
    'CreditHours':'CreditHours__c',
    'BootCampCourse':'BootCampCourse__c',    
}, inplace=True)
course_data_df.head()

,ID_Course__c,CourseCode__c,CourseName__c,CreditHours__c,BootCampCourse__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [49]:
#course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
#course_data_df.head()

In [50]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'ID_Course__c': 1,
  'CourseCode__c': 'BC-DATAVIZ',
  'CourseName__c': 'Data Visualization and Analytics',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 2,
  'CourseCode__c': 'BC-WEBDEV',
  'CourseName__c': 'Full Stack Web Development',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 3,
  'CourseCode__c': 'BC-UIUX',
  'CourseName__c': 'User Interface/User Experience',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 4,
  'CourseCode__c': 'CIS-349',
  'CourseName__c': 'Introduction to Databases',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 5,
  'CourseCode__c': 'CIS-405',
  'CourseName__c': 'Database Programming',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 6,
  'CourseCode__c': 'CIS-438',
  'CourseName__c': 'Database Administration',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 7,
  'CourseCode__c': 'CIS-430',
  'CourseName__c': 'Business Systems Prog

In [52]:
for rec in course_data_records:

    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'CourseCode__c': rec['CourseCode__c'],
        'CourseName__c': rec['CourseName__c'],
        'CreditHours__c': rec['CreditHours__c'],
        'BootCampCourse__c': rec['BootCampCourse__c']
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Bulk 
#sf.bulk.Course__c.insert(course_data_records)

## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [53]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT CourseCode__c, Name FROM Course__c")
for row in data:
    rec = {
        'Course_ID__c': row['Name'], # this is a critical line of code
        'CourseCode__c': row['CourseCode__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'Course_ID__c': 'a004x000003UPYt', 'CourseCode__c': 'BC-WEBDEV'},
 {'Course_ID__c': 'a004x000003UPZD', 'CourseCode__c': 'CIS-438'},
 {'Course_ID__c': 'a004x000003UPZI', 'CourseCode__c': 'CIS-430'},
 {'Course_ID__c': 'a004x000003UPZ3', 'CourseCode__c': 'CIS-349'},
 {'Course_ID__c': 'a004x000003UPYy', 'CourseCode__c': 'BC-UIUX'},
 {'Course_ID__c': 'a004x000003UPYo', 'CourseCode__c': 'BC-DATAVIZ'},
 {'Course_ID__c': 'a004x000003UPZ8', 'CourseCode__c': 'CIS-405'},
 {'Course_ID__c': 'a004x000003UPZN', 'CourseCode__c': 'CIS-435'}]

In [54]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,Course_ID__c,CourseCode__c
0,a004x000003UPYt,BC-WEBDEV
1,a004x000003UPZD,CIS-438
2,a004x000003UPZI,CIS-430
3,a004x000003UPZ3,CIS-349
4,a004x000003UPYy,BC-UIUX
5,a004x000003UPYo,BC-DATAVIZ
6,a004x000003UPZ8,CIS-405
7,a004x000003UPZN,CIS-435


In [55]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        using (ID_Course)

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [57]:
class_data_df.rename(columns={
    'CourseCode':'CourseCode__c',
    'ID_Class':'ID_Class__c',
    'ID_Course':'ID_Course__c',
    'Section':'Section__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c',
   # 'CourseCode':'Course_Code__c'
}, inplace=True)

class_data_df

,CourseCode__c,ID_Class__c,ID_Course__c,Section__c,StartDate__c,EndDate__c
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [58]:
#class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'StartDate__c', 'EndDate__c']]
#class_data_df

## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [59]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

,CourseCode__c,ID_Class__c,ID_Course__c,Section__c,StartDate__c,EndDate__c,Course_ID__c
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003UPYo
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003UPYo
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003UPYt
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003UPYy


In [60]:
class_data_df
class_data_df['StartDate__c'] = pd.to_datetime(class_data_df['StartDate__c']).dt.date
class_data_df['EndDate__c'] = pd.to_datetime(class_data_df['EndDate__c']).dt.date

class_data_df.head()

,CourseCode__c,ID_Class__c,ID_Course__c,Section__c,StartDate__c,EndDate__c,Course_ID__c
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003UPYo
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003UPYo
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003UPYt
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003UPYy


In [61]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'CourseCode__c': 'BC-DATAVIZ',
  'ID_Class__c': 1,
  'ID_Course__c': 1,
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': datetime.date(2020, 3, 14),
  'Course_ID__c': 'a004x000003UPYo'},
 {'CourseCode__c': 'BC-DATAVIZ',
  'ID_Class__c': 2,
  'ID_Course__c': 1,
  'Section__c': 'GWDC201805DATA3',
  'StartDate__c': datetime.date(2018, 5, 15),
  'EndDate__c': datetime.date(2018, 11, 8),
  'Course_ID__c': 'a004x000003UPYo'},
 {'CourseCode__c': 'BC-WEBDEV',
  'ID_Class__c': 3,
  'ID_Course__c': 2,
  'Section__c': 'GWARL201905WEB3',
  'StartDate__c': datetime.date(2019, 5, 14),
  'EndDate__c': datetime.date(2019, 11, 7),
  'Course_ID__c': 'a004x000003UPYt'},
 {'CourseCode__c': 'BC-UIUX',
  'ID_Class__c': 4,
  'ID_Course__c': 3,
  'Section__c': 'GWARL201905UIUX3',
  'StartDate__c': datetime.date(2019, 5, 14),
  'EndDate__c': datetime.date(2019, 11, 7),
  'Course_ID__c': 'a004x000003UPYy'}]

## Insert `Class` Records into Salesforce

In [62]:
for rec in class_data_records:
 
    record = {
        'CourseCode__c': rec['CourseCode__c'],
        'ID_Class__c': rec['ID_Class__c'],
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'StartDate__c': str(rec['StartDate__c']),
        'EndDate__c': str(rec['EndDate__c']),
        'Course_ID__c': rec['Course_ID__c'],
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [33]:
course_records = sf.query("SELECT Id FROM Course__c")
recs_to_delete = [{'Id': r['Id']} for r in course_records['records']]
recs_to_delete

[{'Id': 'a004x000003UOSMAA4'},
 {'Id': 'a004x000003UOSRAA4'},
 {'Id': 'a004x000003UOSWAA4'},
 {'Id': 'a004x000003UOSbAAO'},
 {'Id': 'a004x000003UOSgAAO'},
 {'Id': 'a004x000003UOSlAAO'},
 {'Id': 'a004x000003UOSqAAO'},
 {'Id': 'a004x000003UOSvAAO'}]

In [34]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [35]:
for rec in recs_to_delete:
    try:
        sf.Course__c.delete(rec['Id'])
    except Exception as e:
        print(e)